In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectFromModel
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC
import sklearn
from sklearn.impute import IterativeImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,precision_recall_curve, \
roc_auc_score,roc_curve,recall_score,classification_report, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedShuffleSplit
from lightgbm import LGBMClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import imblearn
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
from imblearn import pipeline

In [2]:
df = pd.read_csv('../input/iba-ml1-mid-project/train.csv')
df['credit_line_utilization'] = df['credit_line_utilization'].str.replace(',', '.').astype(float)
columns = df.iloc[:, 1:-1].columns
columns

Index(['age', 'number_dependent_family_members', 'monthly_income',
       'number_of_credit_lines', 'real_estate_loans',
       'ratio_debt_payment_to_income', 'credit_line_utilization',
       'number_of_previous_late_payments_up_to_59_days',
       'number_of_previous_late_payments_up_to_89_days',
       'number_of_previous_late_payments_90_days_or_more'],
      dtype='object')

In [3]:
X, y = df[columns], df.iloc[:, -1]
Counter(y)

Counter({0: 67148, 1: 5013})

In [7]:
X_imp = SimpleImputer(strategy='median').fit_transform(X)

In [11]:
X_imp = pd.DataFrame(X_imp)
X_imp.columns = columns

In [26]:
ros = SMOTE(sampling_strategy=0.1)
X_rus, y_rus = ros.fit_resample(X_imp, y)
print(Counter(y_rus))
rus = RandomUnderSampler(sampling_strategy=1)
X_rus, y_rus = rus.fit_resample(X_rus, y_rus)
print(Counter(y_rus))

Counter({0: 67148, 1: 6714})
Counter({0: 6714, 1: 6714})


In [35]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X,y)

In [36]:
Counter(y_rus)

Counter({0: 5013, 1: 5013})

In [ ]:
roc = []
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0) 
    
    pipez = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipez.fit(X_train, y_train)
    y_predicted=pipez.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
#     recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, test_size=.3, random_state=0) 
    
    pipe7 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe7.fit(X_train, y_train)
    y_predicted=pipe7.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
#     recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, test_size=.3, random_state=0) 
    
    pipe7 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier())
    ])
    pipe7.fit(X_train, y_train)
    y_predicted=pipe7.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
#     recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe8 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(RandomForestClassifier(n_estimators=200), n_estimators=20))
    ])
    pipe8.fit(X_train, y_train)
    y_predicted=pipe8.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [6]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe8 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe8.fit(X_train, y_train)
    y_predicted=pipe8.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.831526433211733


In [27]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe8 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe8.fit(X_train, y_train)
    y_predicted=pipe8.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8652774330815746


In [22]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe8f = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('f_classif', SelectKBest()),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe8f.fit(X_train, y_train)
    y_predicted=pipe8f.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8311692466118796


In [21]:
?SelectKBest

In [14]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe85 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier())
    ])
    pipe85.fit(X_train, y_train)
    y_predicted=pipe85.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8471894581902826


In [28]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe85 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier())
    ])
    pipe85.fit(X_train, y_train)
    y_predicted=pipe85.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8740216367324336


In [29]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe85 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=GradientBoostingClassifier(), n_estimators=50))
    ])
    pipe85.fit(X_train, y_train)
    y_predicted=pipe85.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8737855095540896


In [22]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe85 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=GradientBoostingClassifier(), n_estimators=50))
    ])
    pipe85.fit(X_train, y_train)
    y_predicted=pipe85.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8753268573677702


In [22]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe85 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=GradientBoostingClassifier(), n_estimators=50))
    ])
    pipe85.fit(X_train, y_train)
    y_predicted=pipe85.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.848941558319215


In [25]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe85 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=ExtraTreesClassifier(), n_estimators=50))
    ])
    pipe85.fit(X_train, y_train)
    y_predicted=pipe85.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8744723303031826


In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe86 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', XGBClassifier())
    ])
    pipe86.fit(X_train, y_train)
    y_predicted=pipe86.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_out, y_out):
    X_train, X_test = X_out.iloc[train_idx], X_out.iloc[test_idx]
    y_train, y_test = y_out.iloc[train_idx], y_out.iloc[test_idx]
    
    pipe82 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', CatBoostClassifier())
    ])
    pipe82.fit(X_train, y_train)
    y_predicted=pipe82.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe81 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', LGBMClassifier())
    ])
    pipe81.fit(X_train, y_train)
    y_predicted=pipe81.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe81 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', XGBClassifier())
    ])
    pipe81.fit(X_train, y_train)
    y_predicted=pipe81.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe81.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
print(np.mean(roc))

In [ ]:
X_sms = KNNImputer().fit_transform(X)
X_sms

In [ ]:
X_sms = SimpleImputer(strategy='mean').fit_transform(X)

In [ ]:
X_sms = pd.DataFrame(X_sms)
X_sms.columns = columns

In [ ]:
X_sms.isna().sum()

In [ ]:
over = SMOTE(sampling_strategy=.1)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeli = pipeline.Pipeline(steps=steps)

In [ ]:
?SMOTE

In [ ]:
X_smt, y_smt = pipeli.fit_resample(X_sms, y)
Counter(y_smt)

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipea4 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipea4.fit(X_train, y_train)
    y_predicted=pipea4.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipea5 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', CatBoostClassifier())
    ])
    pipea5.fit(X_train, y_train)
    y_predicted=pipea5.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe9 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe9.fit(X_train, y_train)
    y_predicted=pipe9.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe9.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe10 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer(n_neighbors=20)),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe10.fit(X_train, y_train)
    y_predicted=pipe10.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe10.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

## PCA GridSearch PolynomialFeature

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
# from sklearn.feature_selection import SelectKBest

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    pipe12 = Pipeline(steps=[
            ('preprocessing', ColumnTransformer(transformers=[
                ('numeric', Pipeline(steps=[
                    ('impute', SimpleImputer(strategy='mean')),
                    ('scale', StandardScaler())
                ]), columns),
            ])),
            ('classifier', BaggingClassifier(base_estimator=DecisionTreeClassifier()))
        ])
    params = {
        'classifier__n_estimators': [80, 100, 120],
    }

    RF_gs = GridSearchCV(pipe12, param_grid=params, scoring='roc_auc')
    RF_gs.fit(X_train, y_train)
    y_predicted=RF_gs.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:,1]))
print(np.mean(roc))

In [ ]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe8 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier())
    ])
    pipe8.fit(X_train, y_train)
    y_predicted=pipe8.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
sss = StratifiedKFold(10)
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe89 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier())
    ])
    pipe89.fit(X_train, y_train)
    
    params = {
        'classifier__learning_rate': [0.1],
#         'classifier__subsample': [0.5, 0.7, 1.0],
#         'classifier__max_depth': [3, 7, 9],
        'classifier__n_estimators': [10, 100, 500]
    }

    RF_gs8 = GridSearchCV(pipe89, param_grid=params, scoring='roc_auc')
    RF_gs8.fit(X_train, y_train)
    
    y_predicted=RF_gs8.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

0.8446206123748242

In [ ]:
GradientBoostingClassifier().get_params()

In [ ]:
?GradientBoostingClassifier

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=RF_gs.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe14 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='median')),
                ('scale', StandardScaler()),
                ('poly', PolynomialFeatures())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe14.fit(X_train, y_train)
    y_predicted=pipe14.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe14.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe15 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', KNNImputer()),
                ('scale', RobustScaler()),
            ]), columns),
        ])),
        ('umap', UMAP(n_components=8)),
        ('classifier',RandomForestClassifier())
    ])
    pipe15.fit(X_train, y_train)
    y_predicted=pipe15.predict_proba(X_test)
    roc.append(roc_auc_score(y_test, y_predicted[:, 1]))
    
print(np.mean(roc))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe15.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [20]:
from umap import UMAP

In [ ]:
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
# X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_rus)

In [ ]:
umap_reducer = umap.UMAP(n_components=2)
train_embedding = umap_reducer.fit_transform(X_imputed[columns])
plt.figure(figsize=(14, 6))
plt.scatter(train_embedding[:, 0], train_embedding[:, 1], s=4, c=y_rus)
plt.show()

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
iso = IsolationForest()
yhat = iso.fit_predict(X_imputed)
mask = yhat != -1
X_iso, y_iso = X_imputed[mask], y_rus[mask]

In [ ]:
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X_rus)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns

In [ ]:
iso = IsolationForest()
ocs = OneClassSVM()
y_iso = iso.fit_predict(X_imputed)
y_ocs = ocs.fit_predict(X_imputed)

m_iso = y_iso != -1
m_ocs = y_ocs != -1

In [ ]:
X_iso, y_iso = X_imputed[m_iso], y_rus[m_iso]
X_ocs, y_ocs = X_imputed[m_ocs], y_rus[m_ocs]

In [ ]:
X_iso.shape

In [ ]:
y_iso.shape

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_iso, y_iso):
    X_train, X_test = X_iso.iloc[train_idx], X_iso.iloc[test_idx]
    y_train, y_test = y_iso.iloc[train_idx], y_iso.iloc[test_idx]
    
    pipe20 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe20.fit(X_train, y_train)
    y_predicted=pipe20.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_iso, y_iso):
    X_train, X_test = X_iso.iloc[train_idx], X_iso.iloc[test_idx]
    y_train, y_test = y_iso.iloc[train_idx], y_iso.iloc[test_idx]
    
    pipe22 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=30))
    ])
    pipe22.fit(X_train, y_train)
    y_predicted=pipe22.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe22.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = pd.concat([X_rus, y_rus], axis=1).corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
?df.corr()

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    pipe30 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', XGBClassifier())
    ])
    pipe30.fit(X_train, y_train)
    y_predicted=pipe30.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

 ### ClassifierChain || pipe model series 40

In [ ]:
from eli5 import show_weights
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import cross_val_score

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (LGBMClassifier()))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe41 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (XGBClassifier()))
    ])
    pipe41.fit(X_train, y_train)
    y_predicted=pipe41.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=20))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe42 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (CatBoostClassifier()))
    ])
    pipe42.fit(X_train, y_train)
    y_predicted=pipe42.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe43 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', RobustScaler())
            ]), columns),
        ])),
        ('classifier', GradientBoostingClassifier(n_estimators=200, learning_rate=0.5,max_depth=1, random_state=0))
    ])
    pipe43.fit(X_train, y_train)
    y_predicted=pipe43.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
cv = KFold(n_splits=8, random_state=1, shuffle=True)
# create model
# model = DecisionTreeClassifier()
# evaluate model
scores = cross_val_score(pipe9, X_rus, y_rus, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy:',np.mean(scores))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe43.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

### Dimensionality reduction || pipe version 50

In [16]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(SimpleImputer(strategy='mean').fit_transform(X_rus))

In [ ]:
cv = StratifiedKFold(n_splits=8, random_state=1, shuffle=True)
# create model
model = DecisionTreeClassifier()
# evaluate model
scores = cross_val_score(pipe9, X_rus, y_rus, scoring='roc_auc', cv=cv, n_jobs=-1)
# report performance
print('Accuracy:',np.mean(scores))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_pca, y_rus):
    X_train, X_test = X_pca[train_idx], X_pca[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe40 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', (LGBMClassifier()))
    ])
    pipe40.fit(X_train, y_train)
    y_predicted=pipe40.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
test = pd.read_csv('../input/iba-ml1-mid-project/test.csv')
test['credit_line_utilization'] = test['credit_line_utilization'].str.replace(',', '.').astype(float)
test_prediction = pipe9z.predict(test.iloc[:, 1:])
submission = pd.DataFrame({'Predicted':test_prediction})
submission = submission.set_index(pd.Index(np.arange(1, 48109))).reset_index()

In [ ]:
submission.to_csv('submission9z.csv')

In [ ]:
perm = PermutationImportance(pipe40, scoring = 'roc_auc'
,random_state=101).fit(X_test, y_test)
show_weights(perm, feature_names = list(X_test.columns))

### Outlier removed || Pipe 60 

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.ensemble import IsolationForest
simple_impute = SimpleImputer(strategy='mean')
X_imputed = simple_impute.fit_transform(X)
X_imputed = pd.DataFrame(X_imputed)
X_imputed.columns = columns
iso = IsolationForest()
yhat = iso.fit_predict(X_imputed)
mask = yhat != -1
X_iso, y_iso = X_imputed[mask], y[mask]

In [ ]:
print(X_imputed.shape)
print(X_iso.shape)

In [ ]:
Counter(y_iso)

In [ ]:
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = imblearn.pipeline.Pipeline(steps=steps)
X_smt, y_smt = pipeline.fit_resample(X_iso, y_iso)
Counter(y_smt)

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipe60 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier())
    ])
    pipe60.fit(X_train, y_train)
    y_predicted=pipe60.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe60.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    pipe61 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', VotingClassifier(estimators=[
            ('forest50', RandomForestClassifier(n_estimators=50)),
            ('forest75', RandomForestClassifier(n_estimators=75)),
            ('forest100', RandomForestClassifier(n_estimators=100)),
            ('forest125', RandomForestClassifier(n_estimators=125)),
            ('forest150', RandomForestClassifier(n_estimators=150)),
        ], voting='hard'))
    ])
    pipe61.fit(X_train, y_train)
    y_predicted=pipe61.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe61.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
sss = StratifiedShuffleSplit()
roc = []
recall = []
for train_idx, test_idx in sss.split(X_smt, y_smt):
    X_train, X_test = X_smt.iloc[train_idx], X_smt.iloc[test_idx]
    y_train, y_test = y_smt.iloc[train_idx], y_smt.iloc[test_idx]
    
    param_grid = {
        'pca__n_components': [2, 4, 5, 9],
        'classifier__n_estimators': [80, 100, 150, 200, 300],
    }
    pipe62 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('pca', PCA()),
        ('classifier', RandomForestClassifier())
    ])
    search = GridSearchCV(pipe62, param_grid, n_jobs=-1)
    search.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
    y_predicted=search.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    recall.append(recall_score(y_test, y_predicted))
    
print(np.mean(roc))
print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
roc = []
recall = []
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus)

param_grid = {
#     'classifier__n_estimators': [100, 200],
#     'tfidf__norm': ['l1','l2']
}
pipe63 = Pipeline(steps=[
    ('preprocessing', ColumnTransformer(transformers=[
        ('numeric', Pipeline(steps=[
            ('impute', KNNImputer()),
            ('scale', StandardScaler())
        ]), columns),
    ])),
#     ('tfidf', TfidfTransformer(norm='l1')),
    ('classifier', RandomForestClassifier(n_estimators=200))
])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
search1 = GridSearchCV(pipe63, param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
search1.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
y_predicted=search1.predict(X_test)
print(roc_auc_score(y_test, y_predicted))
# recall.append(recall_score(y_test, y_predicted))
    
# print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search1.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
roc = []
recall = []
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus)

param_grid = {
#     'classifier__n_estimators': [100, 200],
#     'tfidf__norm': ['l1','l2']
    'pca__n_components': [1, 5]
}
pipe64= Pipeline(steps=[
    ('preprocessing', ColumnTransformer(transformers=[
        ('numeric', Pipeline(steps=[
            ('impute', IterativeImputer()),
            ('scale', RobustScaler())
        ]), columns),
    ])),
    ('pca', PCA()),
    ('classifier', RandomForestClassifier(n_estimators=200))
])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
search2 = GridSearchCV(pipe64, param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
search2.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
y_predicted=search2.predict(X_test)
print(roc_auc_score(y_test, y_predicted))
# recall.append(recall_score(y_test, y_predicted))
    
# print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search2.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
?PCA

In [ ]:
roc = []
recall = []
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus)

param_grid = {
    'classifier__n_estimators': [100,150],
    'pca__n_components': [6, 9],
#     'classifier__learning_rate': [0.01, 0.2, 1],
#     "classifier__min_samples_split": np.linspace(0.1, 0.5, 12),
#     "classifier__min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "preprocessing__numeric__impute__n_neighbors": [10, 15]   
}

pipe65= Pipeline(steps=[
    ('preprocessing', ColumnTransformer(transformers=[
        ('numeric', Pipeline(steps=[
            ('impute', KNNImputer()),
            ('scale', StandardScaler())
        ]), columns),
    ])),
    ('pca', PCA()),
    ('classifier', RandomForestClassifier())
])
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
search3 = GridSearchCV(pipe65, param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
search3.fit(X_train, y_train)
#     pipe62.fit(X_train, y_train)
y_predicted=search3.predict(X_test)
print(roc_auc_score(y_test, y_predicted))
# recall.append(recall_score(y_test, y_predicted))

# print(np.mean(roc))
# print(np.mean(recall))

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=search3.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [ ]:
search3.best_params_

In [ ]:
Counter(y_iso)

In [ ]:
ros = RandomOverSampler(sampling_strategy=0.5)
rus = RandomUnderSampler(sampling_strategy=0.8)

X_ros, y_ros = ros.fit_resample(X, y)
X_comb, y_comb = rus.fit_resample(X_ros, y_ros)

In [ ]:
Counter(y_ros)

In [ ]:
sss = StratifiedKFold()
roc = []
for train_idx, test_idx in sss.split(X_rus, y_rus):
    X_train, X_test = X_rus.iloc[train_idx], X_rus.iloc[test_idx]
    y_train, y_test = y_rus.iloc[train_idx], y_rus.iloc[test_idx]
    
    pipe65 = Pipeline(steps=[
        ('preprocessing', ColumnTransformer(transformers=[
            ('numeric', Pipeline(steps=[
                ('impute', SimpleImputer(strategy='mean')),
                ('scale', StandardScaler())
            ]), columns),
        ])),
        ('classifier', RandomForestClassifier(n_estimators=200))
    ])
    pipe65.fit(X_train, y_train)
    y_predicted=pipe65.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
    
print(np.mean(roc))

In [ ]:
Counter(y_rus)

In [ ]:
roc = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y) 
    y_predicted=pipe65.predict(X_test)
    roc.append(roc_auc_score(y_test, y_predicted))
print(np.mean(roc))

In [30]:
test = pd.read_csv('../input/iba-ml1-mid-project/test.csv')
test['credit_line_utilization'] = test['credit_line_utilization'].str.replace(',', '.').astype(float)
test_prediction = pipe85.predict_proba(test.iloc[:, 1:])
submission = pd.concat([ test['Id'], pd.DataFrame({'Predicted':test_prediction[:, 1]})], axis=1)

In [24]:
submission.to_csv('submission85g.csv', index=False)

In [ ]:
submission

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
!kaggle competitions submit -c iba-ml1-mid-project -f submission7n.csv -m "Message"